In [1]:
from dotenv import load_dotenv

load_dotenv()

from rich.console import Console
console = Console(width=96)

## Setup Tools


In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

In [3]:
from typing import Dict, Any
from tavily import TavilyClient
from langchain.tools import tool

tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

In [4]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")

@tool
def query_playlist_db(query: str) -> str:

    """Query the database for playlist information"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error querying database: {e}"

## Create State

In [5]:
from langchain.agents import AgentState

class WeddingState(AgentState):
    origin: str
    destination: str
    guest_count: str
    genre: str

## Create Subagents


In [6]:
from langchain.agents import create_agent

# Travel agent
travel_agent = create_agent(
    model="gpt-5-nano",
    tools=tools,
    system_prompt="""
    You are a travel agent. Search for flights to the desired destination wedding location.
    You are not allowed to ask any more follow up questions, you must find the best flight options based on the following criteria:
    - Price (lowest, economy class)
    - Duration (shortest)
    - Date (time of year which you believe is best for a wedding at this location)
    To make things easy, only look for one ticket, one way.
    You may need to make multiple searches to iteratively find the best options.
    You will be given no extra information, only the origin and destination. It is your job to think critically about the best options.
    Once you have found the best options, let the user know your shortlist of options.
    """
)

In [7]:
# Venue agent
venue_agent = create_agent(
    model="gpt-5-nano",
    tools=[web_search],
    system_prompt="""
    You are a venue specialist. Search for venues in the desired location, and with the desired capacity.
    You are not allowed to ask any more follow up questions, you must find the best venue options based on the following criteria:
    - Price (lowest)
    - Capacity (exact match)
    - Reviews (highest)
    You may need to make multiple searches to iteratively find the best options.
    """
)

In [8]:
# Playlist agent
playlist_agent = create_agent(
    model="gpt-5-nano",
    tools=[query_playlist_db],
    system_prompt="""
    You are a playlist specialist. Query the sql database and curate the perfect playlist for a wedding given a genre.
    Once you have your playlist, calculate the total duration and cost of the playlist, each song has an associated price.
    If you run into errors when querying the database, try to fix them by making changes to the query.
    Do not come back empty handed, keep trying to query the db until you find a list of songs.
    You may need to make multiple queries to iteratively find the best options.
    """
)

## Main Coordinator


In [9]:
from langchain.tools import ToolRuntime
from langchain.messages import HumanMessage, ToolMessage
from langgraph.types import Command

@tool
async def search_flights(runtime: ToolRuntime) -> str:
    """Travel agent searches for flights to the desired destination wedding location."""
    origin = runtime.state["origin"]
    destination = runtime.state["destination"]
    response = await travel_agent.ainvoke({"messages": [HumanMessage(content=f"Find flights from {origin} to {destination}")]})
    return response['messages'][-1].content

@tool
def search_venues(runtime: ToolRuntime) -> str:
    """Venue agent chooses the best venue for the given location and capacity."""
    destination = runtime.state["destination"]
    capacity = runtime.state["guest_count"]
    query = f"Find wedding venues in {destination} for {capacity} guests"
    response = venue_agent.invoke({"messages": [HumanMessage(content=query)]})
    return response['messages'][-1].content

@tool
def suggest_playlist(runtime: ToolRuntime) -> str:
    """Playlist agent curates the perfect playlist for the given genre."""
    genre = runtime.state["genre"]
    query = f"Find {genre} tracks for wedding playlist"
    response = playlist_agent.invoke({"messages": [HumanMessage(content=query)]})
    return response['messages'][-1].content

@tool
def update_state(origin: str, destination: str, guest_count: str, genre: str, runtime: ToolRuntime) -> str:
    """Update the state when you know all of the values: origin, destination, guest_count, genre"""
    return Command(update={
        "origin": origin, 
        "destination": destination, 
        "guest_count": guest_count, 
        "genre": genre, 
        "messages": [ToolMessage("Successfully updated state", tool_call_id=runtime.tool_call_id)]}
        )


In [10]:
from langchain.agents import create_agent

coordinator = create_agent(
    model="gpt-5-nano",
    tools=[search_flights, search_venues, suggest_playlist, update_state],
    state_schema=WeddingState,
    system_prompt="""
    You are a wedding coordinator. Delegate tasks to your specialists for flights, venues and playlists.
    First find all the information you need to update the state. Once that is done you can delegate the tasks.
    Once you have received their answers, coordinate the perfect wedding for me.
    """
)


## Test


In [11]:
from langchain.messages import HumanMessage

response = await coordinator.ainvoke(
    {
        "messages": [HumanMessage(content="I'm from London and I'd like a wedding in Paris for 100 guests, jazz-genre")],
    }
)

In [12]:
from pprint import pprint

console.print(response)

{
    'messages': [
        HumanMessage(
            content="I'm from London and I'd like a wedding in Paris for 100 guests, 
jazz-genre",
            additional_kwargs={},
            response_metadata={},
            id='ded61052-55e9-47ed-9aa3-369afaf42005'
        ),
        AIMessage(
            content='',
            additional_kwargs={'refusal': None},
            response_metadata={
                'token_usage': {
                    'completion_tokens': 421,
                    'prompt_tokens': 293,
                    'total_tokens': 714,
                    'completion_tokens_details': {
                        'accepted_prediction_tokens': 0,
                        'audio_tokens': 0,
                        'reasoning_tokens': 384,
                        'rejected_prediction_tokens': 0
                    },
                    'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
                },
                'model_provider': 'openai',
                'model_name': 'gpt-5-nano-2025-08-07',
                'system_fingerprint': None,
                'id': 'chatcmpl-CyTxjfDTvtjbjAa9LHD5H8sVkYkxY',
                'service_tier': 'default',
                'finish_reason': 'tool_calls',
                'logprobs': None
            },
            id='lc_run--019bc4a7-ce5a-7d83-96f9-8f84005577ba-0',
            tool_calls=[
                {
                    'name': 'update_state',
                    'args': {
                        'origin': 'London',
                        'destination': 'Paris',
                        'guest_count': '100',
                        'genre': 'jazz'
                    },
                    'id': 'call_ar7cg13AtbTbzAmtT9HKUwCV',
                    'type': 'tool_call'
                }
            ],
            usage_metadata={
                'input_tokens': 293,
                'output_tokens': 421,
                'total_tokens': 714,
                'input_token_details': {'audio': 0, 'cache_read': 0},
                'output_token_details': {'audio': 0, 'reasoning': 384}
            }
        ),
        ToolMessage(
            content='Successfully updated state',
            name='update_state',
            id='c28f88d1-c7cb-4ade-b157-78e6f22fb790',
            tool_call_id='call_ar7cg13AtbTbzAmtT9HKUwCV'
        ),
        AIMessage(
            content='',
            additional_kwargs={'refusal': None},
            response_metadata={
                'token_usage': {
                    'completion_tokens': 446,
                    'prompt_tokens': 338,
                    'total_tokens': 784,
                    'completion_tokens_details': {
                        'accepted_prediction_tokens': 0,
                        'audio_tokens': 0,
                        'reasoning_tokens': 384,
                        'rejected_prediction_tokens': 0
                    },
                    'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
                },
                'model_provider': 'openai',
                'model_name': 'gpt-5-nano-2025-08-07',
                'system_fingerprint': None,
                'id': 'chatcmpl-CyTxnu2GU3LCN9VfkJ1h4PU7AMVKW',
                'service_tier': 'default',
                'finish_reason': 'tool_calls',
                'logprobs': None
            },
            id='lc_run--019bc4a7-e0b6-7d52-9245-25f16b54b5fe-0',
            tool_calls=[
                {
                    'name': 'search_flights',
                    'args': {},
                    'id': 'call_Z00Xw5FoeTqSyk0facJQzOBj',
                    'type': 'tool_call'
                },
                {
                    'name': 'search_venues',
                    'args': {},
                    'id': 'call_7xtXfUSOzWsz6inOIbpR1Bdw',
                    'type': 'tool_call'
                },
                {
                    'name': 'suggest_playlist',
                    'args': {},
                    'id': 'call_Hs

In [13]:
console.print(response["messages"][-1].content)

Fantastic choice: Paris, 100 guests, jazz vibe. I’ve pulled together initial options for travel,
venues, and a starter jazz playlist. I’m ready to coordinate next steps as soon as you pick a 
direction or share preferences.

What I found so far

1) Flights (London → Paris)
- Cheapest direct option: STN → CDG, 01 Jun 2026, 15:35–17:45, 1h10m, 47 EUR (one-way)
  Link: https://on.kiwi.com/YomkR5
- Other affordable direct options (short haul, similar times):
  - LTN → CDG, 01 Jun 2026, 13:55–16:20, 1h25m, 48 EUR
    Link: https://on.kiwi.com/9YTFv2
  - LHR → ORY, 01 Jun 2026, 15:05–17:40, 1h35m, 48 EUR
    Link: https://on.kiwi.com/vTcCTw
  - LTN → CDG, 01 Jun 2026, 19:40–22:00, 1h20m, 49 EUR
    Link: https://on.kiwi.com/4o4MhW

Notes:
- These are one-way options. For a 100-guest wedding, I’ll arrange a group/round-trip block and 
negotiate group rates. I can also expand search to nearby dates (late spring to early summer) 
and multiple London airports if you’re flexible.
- Paris in late spring/early summer is ideal for a wedding weekend—long daylight, pleasant 
weather.

2) Venues near 100 guests (Paris)
- Le Paris (Paris 12e)
  - Capacity: 100 guests
  - Price: 21,000–23,000 EUR for 100 guests (typical package)
  - Link: https://www.1001salles.com/mariage/le-paris-75012.aspx
  - Notes: Clear capacity alignment and published package range.

- Les Maquereaux / La Marina Maquereaux (Paris 13e/16e area)
  - Capacity: around 100 for cocktail-style setups
  - Price: from ~3,050 EUR HT for a full day (plus per-person options)
  - Link: https://www.snapevent.fr/location-de-salle/paris/maquereaux-rive-droite-1106
  - Notes: Strong for a chic, modern vibe with adaptable layouts.

- Café Marguerite (Paris 9e)
  - Capacity: private use for 20–100 people
  - Price: on request (listed as 20–100 in privatisation)
  - Link: https://www.1001salles.com/mariage/cafe-marguerite.aspx
  - Notes: More intimate/romantic, good if you want a cosy reception.

- Domaine Salle des fêtes (Paris 75)
  - Capacity: 100 seated (2nd-floor privatisation)
  - Price: on request
  - Link: https://www.1001salles.com/salle-des-fetes-2-paris-v.html

- Paris Breizh (Paris 12e)
  - Capacity: privatisable up to 70–100 (standing) or around 100 in standing layouts
  - Price: on request
  - Link: https://www.1001salles.com/mariage/paris-breizh.aspx

- Notre Dame Réceptions (Paris 5e)
  - Capacity: around 100 (layout-dependent)
  - Price: on request
  - Link: https://www.1001salles.com/mariage/notre-dame-receptions.aspx

Notes:
- Most Paris venues quote “price on request” for weddings; the Le Paris option gives a clear 
published ballpark, which helps budgeting.
- If you want, I can reach out to 2–3 venues to obtain formal quotes for a 100-person seated 
reception and compare inclusions (catering, furniture, AV, wedding coordination).

3) Jazz playlist (starter)
- 16-track curated set aimed at romantic, smooth jazz suitable for background during 
cocktail/dinner and a classy vibe on the dance floor.
- Tracks include a mix of instrumental and vocal jazz with a mellow to mid-tempo tempo.

Starter playlist outline (sample)
1) Despertar – Aisha Duo
2) O Boto (Bôto) – Antonio Carlos Jobim
3) Portia – Miles Davis
4) End Of Romanticism – Spyro Gyra
5) In Time – Dennis Chambers
6) Paris On Mine – Dennis Chambers
7) Jacob’s Ladder – Incognito
8) Colibri – Incognito
9) Talkin Loud and Saying Nothin – Dennis Chambers
10) Blue Rhythm Fantasy – Gene Krupa
11) Baltimore, DC – Dennis Chambers
12) Compulsion – Miles Davis
13) Heliopolis – Spyro Gyra
14) Gypsy – Incognito
15) E.S.P. – Miles Davis
16) Roll Call – Dennis Chambers

Notes:
- Great for ambiance. I can tailor this for:
  - Cocktail hour: smoother, lighter, vocal jazz
  - Dinner: intimate, orchestral, tasteful
  - Dancing: more upbeat jazz standards or swing
- I can replace tracks or add French jazz standards to give a Parisian flavor.

What I need from you to move forward (quick confirmations)
- Dates and window: Do you have a wedd

link to trace: https://smith.langchain.com/public/7b5fe668-d3e3-4af4-b513-a8cacc0c9e84/r